# PROMETHEUS + Tunix: Teaching Gemma to Show Its Work

## Novel Approach: CIC-Guided Reasoning Traces

**Key Insights from PROMETHEUS Protocol:**
1. **NCD on Traces** - Reward traces that compress similarly to known-good patterns
2. **Basin Center Training** - Converge toward canonical reasoning per domain
3. **Kolmogorov Regularization** - Shorter traces (given correctness) = higher reward
4. **Multi-Domain Focus** - Creative, summarization, science (NOT just math)

**Competition Requirements:**
- Model: Gemma2 2B or Gemma3 1B
- Output: `<reasoning>trace</reasoning><answer>answer</answer>`
- Hardware: TPU v5e-8 (9hr session, 20hr/week)
- Framework: Tunix (JAX-native GRPO)

**Implementation based on:** windmaple's starter notebook (163 upvotes)

In [ ]:
# =============================================================================
# CELL 1: SETUP & INSTALLATION
# =============================================================================
import os
os.environ["HF_HUB_DISABLE_XET"] = "1"

# Install dependencies
!pip install -q kagglehub
!pip install -q ipywidgets
!pip install -q tensorflow
!pip install -q tensorflow_datasets
!pip install -q tensorboardX
!pip install -q transformers
!pip install -q grain
!pip install "google-tunix[prod]==0.1.3"
!pip uninstall -q -y flax
!pip install -U flax
!pip install -q datasets

print("Dependencies installed.")

In [ ]:
# =============================================================================
# CELL 2: IMPORTS
# =============================================================================
import functools
import gc
import os
from pprint import pprint
import re
import random
import csv
import shutil

from flax import nnx
import grain
import humanize
import jax
import jax.numpy as jnp
import kagglehub
import numpy as np
import optax
from orbax import checkpoint as ocp
from pathlib import Path
import qwix
import tensorflow_datasets as tfds
from tqdm.auto import tqdm
from tunix.generate import sampler as sampler_lib
from tunix.generate import tokenizer_adapter as tokenizer_lib
from tunix.models.gemma3 import params
from tunix.models.gemma3 import model
from tunix.rl import rl_cluster as rl_cluster_lib
from tunix.rl.grpo.grpo_learner import GRPOConfig, GRPOLearner
from tunix.rl.rollout import base_rollout
from tunix.sft import metrics_logger
from datasets import load_dataset

# Config
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print(f"JAX devices: {jax.devices()}")

In [ ]:
# =============================================================================
# CELL 3: HYPERPARAMETERS ("Efficiency Build" for TPU v5e-8)
# =============================================================================

# ====== Data ======
TRAIN_DATA_DIR = "./data/train"
TEST_DATA_DIR = "./data/test"
TRAIN_FRACTION = 1.0

# ====== LoRA ======
RANK = 32  # Reduced from 64 for memory
ALPHA = 32.0  # Reduced from 64 for memory

# ====== Sharding ======
MESH = [(1, 4), ("fsdp", "tp")]

# ====== GRPO Generation ======
MAX_PROMPT_LENGTH = 256
TOTAL_GENERATION_STEPS = 512

# ====== TEMPERATURE ANNEALING (PROMETHEUS Insight) ======
# Start HOT for exploration, cool down as we converge
# "pump the brakes harder and harder"
TEMP_START = 1.2       # Aggressive exploration initially
TEMP_END = 0.5         # Stable exploitation at end
TEMP_DECAY = "cosine"  # cosine, linear, or step

# Current temp (will be updated during training)
TEMPERATURE = TEMP_START

TOP_P = 1.0
TOP_K = 50
NUM_GENERATIONS = 4  # G in GRPO algorithm

# ====== GRPO Config ======
NUM_ITERATIONS = 1
BETA = 0.08  # KL divergence penalty
EPSILON = 0.2  # Clipping for stable updates

# ====== Training ======
TRAIN_MICRO_BATCH_SIZE = 2  # Reduced from 4 for memory
NUM_BATCHES = 3738
NUM_TEST_BATCHES = 100
EVAL_EVERY_N_STEPS = 10
NUM_EPOCHS = 1

MAX_STEPS = int(NUM_BATCHES * NUM_ITERATIONS * TRAIN_FRACTION * NUM_EPOCHS)

# ====== AdamW + Warmup + Cosine Scheduler ======
LEARNING_RATE = 3e-6
B1 = 0.9
B2 = 0.99
WEIGHT_DECAY = 0.1
WARMUP_STEPS = 0.1 * MAX_STEPS
MAX_GRAD_NORM = 0.1  # Critical for KL stability

# ====== Checkpointing ======
INTERMEDIATE_CKPT_DIR = "/tmp/content/intermediate_ckpt/"
CKPT_DIR = "/tmp/content/ckpts/"
SAVE_INTERVAL_STEPS = 500
MAX_TO_KEEP = 4

# ====== Inference ======
GENERATION_CONFIGS = {
    "greedy": {"temperature": 1e-4, "top_k": 1, "top_p": 1.0},
    "standard": {"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    "liberal": {"temperature": 0.85, "top_k": 2000, "top_p": 1.0},
}


# ====== TEMPERATURE SCHEDULE FUNCTION ======
def get_temperature(step, max_steps):
    """
    Dynamic temperature annealing.
    Start hot (exploration) -> cool down (exploitation)
    """
    progress = step / max_steps
    
    if TEMP_DECAY == "cosine":
        # Cosine decay: smooth transition
        temp = TEMP_END + 0.5 * (TEMP_START - TEMP_END) * (1 + np.cos(np.pi * progress))
    elif TEMP_DECAY == "linear":
        # Linear decay
        temp = TEMP_START - (TEMP_START - TEMP_END) * progress
    elif TEMP_DECAY == "step":
        # Step decay: drop at 33% and 66%
        if progress < 0.33:
            temp = TEMP_START
        elif progress < 0.66:
            temp = (TEMP_START + TEMP_END) / 2
        else:
            temp = TEMP_END
    else:
        temp = TEMP_START
    
    return max(TEMP_END, temp)


print(f"Config: RANK={RANK}, ALPHA={ALPHA}, LR={LEARNING_RATE}")
print(f"Training: {MAX_STEPS} steps, batch={TRAIN_MICRO_BATCH_SIZE}")
print(f"\n🔥 TEMPERATURE ANNEALING:")
print(f"   Start: {TEMP_START} (HOT - exploration)")
print(f"   End:   {TEMP_END} (COOL - exploitation)")
print(f"   Decay: {TEMP_DECAY}")
print(f"\n   Step 0:    temp={get_temperature(0, MAX_STEPS):.2f}")
print(f"   Step 50%:  temp={get_temperature(MAX_STEPS//2, MAX_STEPS):.2f}")
print(f"   Step 100%: temp={get_temperature(MAX_STEPS, MAX_STEPS):.2f}")

In [ ]:
# =============================================================================
# CELL 4: TEMPLATE & SPECIAL TOKENS (Competition Format)
# =============================================================================

# Required format tags (locked by competition)
reasoning_start = "<reasoning>"
reasoning_end = "</reasoning>"
solution_start = "<answer>"
solution_end = "</answer>"

# PROMETHEUS Insight: Multi-domain system prompt
# NOT just math - creative, summarization, science
SYSTEM_PROMPT = f"""You are given a problem. Think about the problem and \
provide your reasoning. Place it between {reasoning_start} and \
{reasoning_end}. Then, provide the final answer between {solution_start} \
and {solution_end}."""

# Gemma3 chat template
TEMPLATE = """<start_of_turn>user
{system_prompt}

{question}<end_of_turn>
<start_of_turn>model"""

print("Template configured.")
print(f"Format: {reasoning_start}...{reasoning_end} then {solution_start}...{solution_end}")

In [ ]:
# =============================================================================
# CELL 5: DATA LOADING (Real User Tasks - PROMETHEUS Insight)
# =============================================================================
# Key insight: Train on what users ACTUALLY ask LLMs
# NOT academic benchmarks - real conversations
#
# Datasets:
# - OpenAssistant (oasst1) - Human-written conversations
# - WildChat - Organic ChatGPT queries
# - ShareGPT/Vicuna - Real user interactions
# - Dolly - Databricks instruction dataset

def get_real_user_dataset(max_samples=10000) -> grain.MapDataset:
    """Load real user conversation datasets."""
    
    all_questions = []
    
    # 1. OpenAssistant (high quality human conversations)
    print("Loading OpenAssistant...")
    try:
        oasst = load_dataset("OpenAssistant/oasst1", split="train")
        # Filter for initial user messages (prompter turns)
        for item in oasst:
            if item.get("role") == "prompter" and item.get("parent_id") is None:
                text = item.get("text", "").strip()
                if 20 < len(text) < 500:  # Reasonable length
                    all_questions.append({
                        "question": text,
                        "source": "oasst",
                        "answer": None  # No ground truth for open-ended
                    })
        print(f"  -> {len([q for q in all_questions if q['source']=='oasst'])} from OpenAssistant")
    except Exception as e:
        print(f"  -> OpenAssistant failed: {e}")
    
    # 2. Dolly (Databricks instructions)
    print("Loading Dolly...")
    try:
        dolly = load_dataset("databricks/databricks-dolly-15k", split="train")
        for item in dolly:
            instruction = item.get("instruction", "").strip()
            context = item.get("context", "").strip()
            if context:
                question = f"{instruction}\n\nContext: {context}"
            else:
                question = instruction
            if 20 < len(question) < 500:
                all_questions.append({
                    "question": question,
                    "source": "dolly",
                    "answer": None
                })
        print(f"  -> {len([q for q in all_questions if q['source']=='dolly'])} from Dolly")
    except Exception as e:
        print(f"  -> Dolly failed: {e}")
    
    # 3. Alpaca (Stanford instruction dataset)
    print("Loading Alpaca...")
    try:
        alpaca = load_dataset("tatsu-lab/alpaca", split="train")
        for item in alpaca:
            instruction = item.get("instruction", "").strip()
            inp = item.get("input", "").strip()
            if inp:
                question = f"{instruction}\n\nInput: {inp}"
            else:
                question = instruction
            if 20 < len(question) < 500:
                all_questions.append({
                    "question": question,
                    "source": "alpaca",
                    "answer": None
                })
        print(f"  -> {len([q for q in all_questions if q['source']=='alpaca'])} from Alpaca")
    except Exception as e:
        print(f"  -> Alpaca failed: {e}")
    
    # 4. FLAN (diverse tasks)
    print("Loading FLAN subset...")
    try:
        flan = load_dataset("Muennighoff/flan", split="train", streaming=True)
        flan_count = 0
        for item in flan:
            if flan_count >= 5000:  # Limit FLAN samples
                break
            question = item.get("inputs", "").strip()
            if 20 < len(question) < 500:
                all_questions.append({
                    "question": question,
                    "source": "flan",
                    "answer": None
                })
                flan_count += 1
        print(f"  -> {flan_count} from FLAN")
    except Exception as e:
        print(f"  -> FLAN failed: {e}")
    
    # Shuffle and limit
    random.shuffle(all_questions)
    all_questions = all_questions[:max_samples]
    
    print(f"\nTotal: {len(all_questions)} real user questions")
    
    # Show distribution
    sources = {}
    for q in all_questions:
        src = q["source"]
        sources[src] = sources.get(src, 0) + 1
    print(f"Distribution: {sources}")
    
    # Convert to grain dataset
    dataset = (
        grain.MapDataset.source(all_questions)
        .shuffle(seed=42)
        .map(
            lambda x: {
                "prompts": TEMPLATE.format(
                    system_prompt=SYSTEM_PROMPT,
                    question=x["question"],
                ),
                "question": x["question"],
                "answer": x["answer"],  # None for open-ended
                "source": x["source"],
            }
        )
    )
    return dataset


# Load datasets
print("Loading REAL USER datasets...")
print("=" * 50)

full_dataset = get_real_user_dataset(max_samples=NUM_BATCHES * TRAIN_MICRO_BATCH_SIZE * 2)

# Split into train/test
dataset = full_dataset.batch(TRAIN_MICRO_BATCH_SIZE)[:NUM_BATCHES]

if TRAIN_FRACTION == 1.0:
    train_dataset = dataset.repeat(NUM_EPOCHS)
    val_dataset = None
else:
    train_dataset = dataset[: int(len(dataset) * TRAIN_FRACTION)]
    train_dataset = train_dataset.repeat(NUM_EPOCHS)
    val_dataset = dataset[int(len(dataset) * TRAIN_FRACTION) :].repeat(NUM_EPOCHS)

# Use last 10% as test
test_start = int(len(dataset) * 0.9)
test_dataset = full_dataset.batch(TRAIN_MICRO_BATCH_SIZE)[test_start:test_start + NUM_TEST_BATCHES]

print(f"\nTrain batches: {len(train_dataset)}")
print(f"Test batches: {len(test_dataset)}")

In [ ]:
# =============================================================================
# CELL 6: LOAD MODEL (Gemma3 1B-IT)
# =============================================================================

def show_hbm_usage():
    """Display memory usage per device."""
    fmt_size = functools.partial(humanize.naturalsize, binary=True)
    for d in jax.local_devices():
        stats = d.memory_stats()
        used = stats["bytes_in_use"]
        limit = stats["bytes_limit"]
        print(f"Using {fmt_size(used)} / {fmt_size(limit)} ({used/limit:%}) on {d}")


# Kaggle authentication
from kaggle_secrets import UserSecretsClient

def auto_login():
    try:
        user_secrets = UserSecretsClient()
        username = user_secrets.get_secret("KAGGLE_USERNAME")
        key = user_secrets.get_secret("KAGGLE_KEY")
        if username and key:
            os.environ["KAGGLE_USERNAME"] = username
            os.environ["KAGGLE_KEY"] = key
            print("✅ Authenticated via Secrets")
            return
    except Exception:
        pass
    
    if "KAGGLE_USERNAME" in os.environ and "KAGGLE_KEY" in os.environ:
        print("✅ Environment variables already set")
        return
    
    print("⚠️ Manual login required...")
    kagglehub.login()

auto_login()

# Clean up old checkpoints
!rm /tmp/content/intermediate_ckpt/* -rf
!rm /tmp/content/ckpts/* -rf

# Load Gemma3 1B-IT
print("Loading Gemma3 1B-IT...")
MODEL_CP_PATH = params.GEMMA3_1B_IT
config = model.ModelConfig.gemma3_1b()
gemma = params.create_model_from_checkpoint(MODEL_CP_PATH, config)
tokenizer = params.create_tokenizer()

# Save intermediate checkpoint for LoRA loading
checkpointer = ocp.StandardCheckpointer()
_, state = nnx.split(gemma)
checkpointer.save(os.path.join(INTERMEDIATE_CKPT_DIR, "state"), state)
checkpointer.wait_until_finished()

# Clean up to save memory
del gemma
del state
gc.collect()

print("Model checkpoint saved.")
show_hbm_usage()

In [ ]:
# =============================================================================
# CELL 7: LORA MODEL SETUP
# =============================================================================

def get_gemma_ref_model(ckpt_path):
    """Load reference model from checkpoint with sharding."""
    mesh = jax.make_mesh(*MESH)
    model_config = model.ModelConfig.gemma3_1b()
    
    abs_gemma = nnx.eval_shape(
        lambda: params.create_model_from_checkpoint(MODEL_CP_PATH, config)
    )

    abs_state = nnx.state(abs_gemma)
    abs_state = jax.tree.map(
        lambda a, s: jax.ShapeDtypeStruct(a.shape, jnp.bfloat16, sharding=s),
        abs_state,
        nnx.get_named_sharding(abs_state, mesh),
    )
    
    checkpointer = ocp.StandardCheckpointer()
    restored_params = checkpointer.restore(ckpt_path, target=abs_state)

    graph_def, _ = nnx.split(abs_gemma)
    gemma = nnx.merge(graph_def, restored_params)
    return gemma, mesh, model_config


def get_lora_model(base_model, mesh):
    """Apply LoRA layers to base model."""
    lora_provider = qwix.LoraProvider(
        module_path=(
            ".*q_einsum|.*kv_einsum|.*gate_proj|.*down_proj|.*up_proj|"
            ".*attn_vec_einsum"
        ),
        rank=RANK,
        alpha=ALPHA,
    )

    model_input = base_model.get_model_input()
    lora_model = qwix.apply_lora_to_model(
        base_model, lora_provider, **model_input
    )

    with mesh:
        state = nnx.state(lora_model)
        pspecs = nnx.get_partition_spec(state)
        sharded_state = jax.lax.with_sharding_constraint(state, pspecs)
        nnx.update(lora_model, sharded_state)

    return lora_model


# Load models
print("Loading reference model...")
ref_model, mesh, model_config = get_gemma_ref_model(
    ckpt_path=os.path.join(INTERMEDIATE_CKPT_DIR, "state")
)

print("Applying LoRA...")
lora_policy = get_lora_model(ref_model, mesh=mesh)

print(f"LoRA model ready: RANK={RANK}, ALPHA={ALPHA}")
show_hbm_usage()

In [ ]:
# =============================================================================
# CELL 8: PROMETHEUS REWARD FUNCTIONS (Open-Ended Tasks)
# =============================================================================
# For real user tasks, we can't check "correctness" - instead reward:
# 1. Format compliance (exact and approximate)
# 2. Reasoning trace quality (coherence, structure, relevance)
# 3. Answer completeness (non-empty, reasonable length)
# 4. Compression bonus (Kolmogorov: concise but complete)

# Format matching regex
match_format = re.compile(
    rf"^[\s]{{0,}}"
    rf"{reasoning_start}.+?{reasoning_end}.*?"
    rf"{solution_start}(.+?){solution_end}"
    rf"[\s]{{0,}}$",
    flags=re.MULTILINE | re.DOTALL,
)


def match_format_exactly(prompts, completions, **kwargs):
    """Reward 3.0 if format matches exactly."""
    return [
        0 if match_format.search(response) is None else 3.0
        for response in completions
    ]


def match_format_approximately(prompts, completions, **kwargs):
    """Reward partial format compliance."""
    scores = []
    for completion in completions:
        score = 0
        response = completion
        # Reward each tag appearing exactly once
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end) == 1 else -0.5
        score += 0.5 if response.count(solution_start) == 1 else -0.5
        score += 0.5 if response.count(solution_end) == 1 else -0.5
        scores.append(score)
    return scores


def reasoning_coherence(prompts, completions, **kwargs):
    """Reward coherent, structured reasoning traces."""
    scores = []
    
    # Logical connectors that indicate good reasoning
    connectors = [
        'first', 'second', 'third', 'finally',
        'because', 'since', 'therefore', 'thus', 'so',
        'however', 'although', 'but', 'yet',
        'for example', 'specifically', 'in particular',
        'this means', 'which means', 'as a result',
        'let me', 'i need to', 'to do this',
        'step', '1.', '2.', '3.',
    ]
    
    for completion in completions:
        # Extract reasoning
        match = re.search(
            rf"{reasoning_start}(.+?){reasoning_end}",
            completion,
            re.DOTALL
        )
        if not match:
            scores.append(0)
            continue
        
        reasoning = match.group(1).lower()
        score = 0
        
        # Count logical connectors (max 2.0 points)
        connector_count = sum(1 for c in connectors if c in reasoning)
        score += min(2.0, connector_count * 0.3)
        
        # Has multiple sentences (max 1.0 point)
        sentences = [s.strip() for s in reasoning.split('.') if len(s.strip()) > 10]
        if len(sentences) >= 3:
            score += 1.0
        elif len(sentences) >= 2:
            score += 0.5
        
        scores.append(score)
    
    return scores


def answer_completeness(prompts, completions, **kwargs):
    """Reward non-empty, substantive answers."""
    scores = []
    
    for completion in completions:
        # Extract answer
        match = re.search(
            rf"{solution_start}(.+?){solution_end}",
            completion,
            re.DOTALL
        )
        if not match:
            scores.append(0)
            continue
        
        answer = match.group(1).strip()
        word_count = len(answer.split())
        
        # Reward based on answer length (not too short, not too long)
        if 5 <= word_count <= 100:
            scores.append(2.0)
        elif 3 <= word_count < 5 or 100 < word_count <= 200:
            scores.append(1.0)
        elif 1 <= word_count < 3:
            scores.append(0.5)
        else:
            scores.append(0)
    
    return scores


def question_relevance(prompts, completions, **kwargs):
    """Reward reasoning that references the question."""
    questions = kwargs.get("question", [""])
    scores = []
    
    for completion, question in zip(completions, questions):
        # Extract reasoning
        match = re.search(
            rf"{reasoning_start}(.+?){reasoning_end}",
            completion,
            re.DOTALL
        )
        if not match:
            scores.append(0)
            continue
        
        reasoning = match.group(1).lower()
        question_lower = question.lower()
        
        # Extract key words from question (nouns, verbs - simple heuristic)
        question_words = set(
            w for w in re.findall(r'\b\w{4,}\b', question_lower)
            if w not in {'what', 'when', 'where', 'which', 'that', 'this', 'have', 'does', 'will', 'would', 'could', 'should'}
        )
        
        if not question_words:
            scores.append(1.0)  # Can't check, give benefit of doubt
            continue
        
        # Count how many question words appear in reasoning
        matches = sum(1 for w in question_words if w in reasoning)
        relevance = matches / len(question_words)
        
        scores.append(min(2.0, relevance * 2.5))
    
    return scores


def trace_compression_bonus(prompts, completions, **kwargs):
    """Kolmogorov bonus: reward concise but complete reasoning."""
    scores = []
    
    for completion in completions:
        # Extract reasoning
        match = re.search(
            rf"{reasoning_start}(.+?){reasoning_end}",
            completion,
            re.DOTALL
        )
        if not match:
            scores.append(0)
            continue
        
        reasoning = match.group(1)
        word_count = len(reasoning.split())
        
        # Optimal range: 30-100 words (concise but substantive)
        if 30 <= word_count <= 100:
            scores.append(1.5)
        elif 20 <= word_count < 30 or 100 < word_count <= 150:
            scores.append(1.0)
        elif 15 <= word_count < 20 or 150 < word_count <= 200:
            scores.append(0.5)
        else:
            scores.append(0)  # Too short or too long
    
    return scores


# Debug helper
def debug_sample(prompts, completions, **kwargs):
    """Print first sample for debugging (returns 0 reward)."""
    questions = kwargs.get("question", [""])
    if completions:
        q = questions[0][:60] if questions else "?"
        c = completions[0][:200] if completions[0] else "(empty)"
        print(f"Q: {q}...")
        print(f"R: {c}...")
    return [0] * len(completions)


print("PROMETHEUS reward functions for OPEN-ENDED tasks:")
print("  - match_format_exactly (3.0)")
print("  - match_format_approximately (+/-0.5 per tag)")
print("  - reasoning_coherence (connectors, structure)")
print("  - answer_completeness (length, substance)")
print("  - question_relevance (keywords from question)")
print("  - trace_compression_bonus (Kolmogorov)")
print("\nNo 'correctness' check - we reward PROCESS not ANSWER")

In [ ]:
# =============================================================================
# CELL 9: EVALUATION HELPERS (Open-Ended Tasks)
# =============================================================================

def generate(question, sampler, temperature=0.7, top_k=50, top_p=0.95, seed=None):
    """Generate response for a question."""
    if isinstance(question, str):
        input_batch = [
            TEMPLATE.format(system_prompt=SYSTEM_PROMPT, question=question),
        ]
    else:
        input_batch = [
            TEMPLATE.format(system_prompt=SYSTEM_PROMPT, question=q)
            for q in question
        ]

    out_data = sampler(
        input_strings=input_batch,
        max_generation_steps=768,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        echo=False,
        seed=seed,
        eos_tokens=[1, 106],
    )

    output = out_data.text
    return output[0] if isinstance(question, str) else output


def evaluate_open_ended(dataset, sampler, temperature=0.7, top_k=50, top_p=0.95, num_passes=1):
    """
    Evaluate on open-ended tasks.
    Since no ground truth, measure:
    - Format compliance
    - Reasoning quality (coherence, length)
    - Answer presence
    """
    total = 0
    format_exact = 0
    has_reasoning = 0
    has_answer = 0
    avg_reasoning_len = []
    avg_answer_len = []
    coherence_scores = []
    
    for batch in tqdm(dataset):
        questions = batch["question"]
        
        responses = generate(questions, sampler, temperature, top_k, top_p, seed=0)
        
        for response in responses:
            total += 1
            
            # Check format
            if match_format.search(response) is not None:
                format_exact += 1
            
            # Check reasoning
            r_match = re.search(
                rf"{reasoning_start}(.+?){reasoning_end}",
                response,
                re.DOTALL
            )
            if r_match:
                has_reasoning += 1
                reasoning = r_match.group(1)
                avg_reasoning_len.append(len(reasoning.split()))
                
                # Simple coherence: count logical connectors
                connectors = ['because', 'therefore', 'first', 'then', 'so', 'thus']
                coherence = sum(1 for c in connectors if c in reasoning.lower())
                coherence_scores.append(coherence)
            
            # Check answer
            a_match = re.search(
                rf"{solution_start}(.+?){solution_end}",
                response,
                re.DOTALL
            )
            if a_match:
                has_answer += 1
                answer = a_match.group(1)
                avg_answer_len.append(len(answer.split()))
    
    return {
        "total": total,
        "format_exact_pct": format_exact / total * 100 if total else 0,
        "has_reasoning_pct": has_reasoning / total * 100 if total else 0,
        "has_answer_pct": has_answer / total * 100 if total else 0,
        "avg_reasoning_words": np.mean(avg_reasoning_len) if avg_reasoning_len else 0,
        "avg_answer_words": np.mean(avg_answer_len) if avg_answer_len else 0,
        "avg_coherence": np.mean(coherence_scores) if coherence_scores else 0,
    }


def print_eval_results(results, label=""):
    """Pretty print evaluation results."""
    print(f"\n{'='*50}")
    print(f"{label} RESULTS")
    print(f"{'='*50}")
    print(f"  Samples evaluated: {results['total']}")
    print(f"\n  FORMAT:")
    print(f"    Exact format match: {results['format_exact_pct']:.1f}%")
    print(f"    Has <reasoning>:    {results['has_reasoning_pct']:.1f}%")
    print(f"    Has <answer>:       {results['has_answer_pct']:.1f}%")
    print(f"\n  QUALITY:")
    print(f"    Avg reasoning words: {results['avg_reasoning_words']:.1f}")
    print(f"    Avg answer words:    {results['avg_answer_words']:.1f}")
    print(f"    Avg coherence score: {results['avg_coherence']:.2f}")


print("Open-ended evaluation helpers defined.")
print("Metrics: format compliance, reasoning presence, quality indicators")

In [ ]:
# =============================================================================
# CELL 10: PRE-TRAINING EVALUATION
# =============================================================================

print("Creating sampler for evaluation...")
sampler = sampler_lib.Sampler(
    transformer=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

print("\nEvaluating BASE model (before training)...")
base_results = evaluate_open_ended(
    test_dataset,
    sampler,
    **GENERATION_CONFIGS["greedy"],
)
print_eval_results(base_results, "BASE MODEL")

In [ ]:
# =============================================================================
# CELL 11: GRPO TRAINING SETUP
# =============================================================================

# Checkpoint options
checkpointing_options = ocp.CheckpointManagerOptions(
    save_interval_steps=SAVE_INTERVAL_STEPS, max_to_keep=MAX_TO_KEEP
)

# Optimizer: AdamW with warmup cosine decay
optimizer = optax.adamw(
    learning_rate=optax.schedules.warmup_cosine_decay_schedule(
        init_value=0.0,
        peak_value=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        decay_steps=MAX_STEPS,
        end_value=0.0,
    ),
    b1=B1,
    b2=B2,
    weight_decay=WEIGHT_DECAY,
)

# Gradient clipping (critical for KL stability)
if MAX_GRAD_NORM is not None:
    optimizer = optax.chain(
        optax.clip_by_global_norm(max_norm=MAX_GRAD_NORM),
        optimizer,
    )

# Training cluster config
cluster_config = rl_cluster_lib.ClusterConfig(
    role_to_mesh={
        rl_cluster_lib.Role.ACTOR: mesh,
        rl_cluster_lib.Role.REFERENCE: mesh,
        rl_cluster_lib.Role.ROLLOUT: mesh,
    },
    rollout_engine='vanilla',
    offload_to_cpu=False,
    training_config=rl_cluster_lib.RLTrainingConfig(
        actor_optimizer=optimizer,
        eval_every_n_steps=EVAL_EVERY_N_STEPS,
        max_steps=MAX_STEPS,
        mini_batch_size=TRAIN_MICRO_BATCH_SIZE,
        train_micro_batch_size=TRAIN_MICRO_BATCH_SIZE,
        metrics_logging_options=None,  # Disable wandb
        checkpoint_root_directory=CKPT_DIR,
        checkpointing_options=checkpointing_options,
    ),
    rollout_config=base_rollout.RolloutConfig(
        max_tokens_to_generate=TOTAL_GENERATION_STEPS,
        max_prompt_length=MAX_PROMPT_LENGTH,
        kv_cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        top_k=TOP_K,
        eos_tokens=[1, 106],
    ),
)

# GRPO config
grpo_config = GRPOConfig(
    num_generations=NUM_GENERATIONS,
    num_iterations=NUM_ITERATIONS,
    beta=BETA,
    epsilon=EPSILON,
)

print("GRPO training config ready.")
print(f"  Steps: {MAX_STEPS}")
print(f"  LR: {LEARNING_RATE} (warmup {int(WARMUP_STEPS)} steps)")
print(f"  GRPO: G={NUM_GENERATIONS}, β={BETA}, ε={EPSILON}")

In [ ]:
# =============================================================================
# CELL 12: INITIALIZE GRPO TRAINER
# =============================================================================

# Create RL cluster
rl_cluster = rl_cluster_lib.RLCluster(
    actor=lora_policy,
    reference=ref_model,
    tokenizer=tokenizer,
    cluster_config=cluster_config,
)

# Create GRPO trainer with PROMETHEUS reward functions for OPEN-ENDED tasks
grpo_trainer = GRPOLearner(
    rl_cluster=rl_cluster,
    reward_fns=[
        match_format_exactly,        # 3.0 for correct format
        match_format_approximately,  # +/-0.5 per tag
        reasoning_coherence,         # Up to 3.0 for logical structure
        answer_completeness,         # Up to 2.0 for substantive answer
        question_relevance,          # Up to 2.0 for staying on topic
        trace_compression_bonus,     # Up to 1.5 for concise reasoning
    ],
    grpo_config=grpo_config,
)

print("GRPO trainer initialized with 6 OPEN-ENDED reward functions:")
print("  Format: exact (3.0) + approximate (+/-2.0)")
print("  Quality: coherence (3.0) + completeness (2.0)")  
print("  Relevance: keywords (2.0) + compression (1.5)")
print(f"\nMax possible reward: ~14.0")
print("Ready to train on REAL USER TASKS!")

In [ ]:
# =============================================================================
# CELL 13: TRAIN!
# =============================================================================

print(f"Starting GRPO training for {MAX_STEPS} steps...")
print(f"This may take several hours on TPU v5e-8.")
print()

with mesh:
    grpo_trainer.train(train_dataset)

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)

In [ ]:
# =============================================================================
# CELL 14: LOAD BEST CHECKPOINT & EVALUATE
# =============================================================================

# Find latest checkpoint
actor_ckpt_dir = os.path.join(CKPT_DIR, "actor")

latest_step = -1
if os.path.exists(actor_ckpt_dir):
    for item in os.listdir(actor_ckpt_dir):
        if os.path.isdir(os.path.join(actor_ckpt_dir, item)) and re.match(r'^\d+$', item):
            step = int(item)
            if step > latest_step:
                latest_step = step

if latest_step == -1:
    raise FileNotFoundError(f"No checkpoints found in {actor_ckpt_dir}")

print(f"Loading checkpoint from step {latest_step}...")

trained_ckpt_path = os.path.join(
    CKPT_DIR, "actor", str(latest_step), "model_params"
)

abs_params = jax.tree.map(
    lambda x: jax.ShapeDtypeStruct(x.shape, x.dtype),
    nnx.state(lora_policy, nnx.LoRAParam),
)
checkpointer = ocp.StandardCheckpointer()
trained_lora_params = checkpointer.restore(trained_ckpt_path, target=abs_params)

nnx.update(
    lora_policy,
    jax.tree.map(
        lambda a, b: b,
        nnx.state(lora_policy, nnx.LoRAParam),
        trained_lora_params,
    ),
)

# Create sampler with trained model
sampler = sampler_lib.Sampler(
    transformer=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

print("\nEvaluating TRAINED model...")
trained_results = evaluate_open_ended(
    test_dataset,
    sampler,
    **GENERATION_CONFIGS["greedy"],
)
print_eval_results(trained_results, "TRAINED MODEL")

# Compare
print("\n" + "="*50)
print("IMPROVEMENT SUMMARY")
print("="*50)
print(f"  Format: {base_results['format_exact_pct']:.1f}% -> {trained_results['format_exact_pct']:.1f}%")
print(f"  Reasoning: {base_results['has_reasoning_pct']:.1f}% -> {trained_results['has_reasoning_pct']:.1f}%")
print(f"  Coherence: {base_results['avg_coherence']:.2f} -> {trained_results['avg_coherence']:.2f}")

In [ ]:
# =============================================================================
# CELL 15: INFERENCE EXAMPLES
# =============================================================================

# Test on various domains (PROMETHEUS: multi-domain focus)
test_questions = [
    # Math (low weight but test anyway)
    "Janet has 3 times as many marbles as Tom. Tom has 12 marbles. How many marbles does Janet have?",
    # Science
    "Why does ice float on water?",
    # Creative
    "Come up with 3 creative uses for a broken umbrella.",
    # Logic
    "If all cats are mammals and some mammals can swim, can we conclude that some cats can swim?",
]

print("="*60)
print("INFERENCE EXAMPLES")
print("="*60)

# Use "standard" config for coherent outputs (greedy with temp=1e-4 produces garbage)
for q in test_questions:
    print(f"\nQ: {q}")
    print("-"*40)
    response = generate(q, sampler, **GENERATION_CONFIGS["standard"])
    print(response)
    print()

In [ ]:
# =============================================================================
# CELL 16: SAVE FOR MULTI-SESSION MODE
# =============================================================================

# For single-session mode: Model is reproduced from this notebook
# For multi-session mode: Upload to Kaggle Models

# MUST use absolute path for Orbax checkpointer
SAVE_PATH = "/kaggle/working/prometheus_gemma_reasoning"

print(f"Saving trained LoRA weights to {SAVE_PATH}...")

# Save LoRA params
os.makedirs(SAVE_PATH, exist_ok=True)
checkpointer = ocp.StandardCheckpointer()
checkpointer.save(
    os.path.join(SAVE_PATH, "lora_params"),
    nnx.state(lora_policy, nnx.LoRAParam)
)
checkpointer.wait_until_finished()

print("\n" + "="*60)
print("PROMETHEUS + TUNIX TRAINING COMPLETE")
print("="*60)
print(f"\nModel saved to: {SAVE_PATH}")
print("\nFor multi-session evaluation:")
print("  1. Upload to Kaggle Models")
print("  2. Specify KAGGLE_MODEL_ID in submission notebook")
print("\nKey settings used:")
print(f"  - Model: Gemma3 1B-IT")
print(f"  - LoRA: RANK={RANK}, ALPHA={ALPHA}")
print(f"  - GRPO: G={NUM_GENERATIONS}, β={BETA}, ε={EPSILON}")
print(f"  - Training: {MAX_STEPS} steps, LR={LEARNING_RATE}")

In [ ]:
# =============================================================================
# CELL 17: INFERENCE SERVER FOR KAGGLE SUBMISSION
# =============================================================================
# This cell sets up the inference server required for Kaggle automated evaluation.
# The server is only activated during competition rerun (KAGGLE_IS_COMPETITION_RERUN=True)

import os
import sys

# Check if we're in competition rerun mode
IS_COMPETITION_RERUN = bool(os.getenv('KAGGLE_IS_COMPETITION_RERUN'))

print(f"Competition rerun mode: {IS_COMPETITION_RERUN}")

# Define the predict function
def predict(question: str) -> str:
    """
    Generate a response with reasoning trace for the given question.
    Returns format: <reasoning>...</reasoning><answer>...</answer>
    """
    response = generate(question, sampler, **GENERATION_CONFIGS["standard"])
    return response


# Try to find and use the inference server
if IS_COMPETITION_RERUN:
    print("Setting up inference server for submission...")
    
    # List available inputs to find the inference server module
    input_dir = "/kaggle/input"
    if os.path.exists(input_dir):
        print(f"Available inputs: {os.listdir(input_dir)}")
        
        # Look for kaggle_evaluation in competition data
        for item in os.listdir(input_dir):
            item_path = os.path.join(input_dir, item)
            if os.path.isdir(item_path):
                # Check for kaggle_evaluation subdirectory
                eval_path = os.path.join(item_path, "kaggle_evaluation")
                if os.path.exists(eval_path):
                    print(f"Found kaggle_evaluation at: {eval_path}")
                    sys.path.insert(0, item_path)
                    break
    
    # Try to import the inference server
    try:
        # First, try Tunix-specific inference server (if it exists)
        try:
            from kaggle_evaluation.tunix_inference_server import TunixInferenceServer
            inference_server = TunixInferenceServer(predict)
            print("Using TunixInferenceServer")
        except ImportError:
            # Try generic inference server
            try:
                from kaggle_evaluation import make_env
                inference_server = make_env(predict)
                print("Using generic make_env")
            except ImportError:
                # Try other common patterns
                import kaggle_evaluation
                # List what's available
                print(f"kaggle_evaluation contents: {dir(kaggle_evaluation)}")
                
                # Try to find any inference server class
                for name in dir(kaggle_evaluation):
                    if 'inference' in name.lower() or 'server' in name.lower():
                        print(f"Found: {name}")
                        cls = getattr(kaggle_evaluation, name)
                        if callable(cls):
                            inference_server = cls(predict)
                            break
        
        # Start serving
        print("Starting inference server...")
        inference_server.serve()
        
    except Exception as e:
        print(f"ERROR: Could not set up inference server: {e}")
        print("Available modules in kaggle_evaluation:")
        try:
            import kaggle_evaluation
            print(dir(kaggle_evaluation))
        except ImportError as ie:
            print(f"kaggle_evaluation not found: {ie}")
        
        # Provide debugging info
        print("\nTo fix this, check the competition data for the correct inference server module.")
        print("Run this in a Kaggle notebook:")
        print("  import os")
        print("  for root, dirs, files in os.walk('/kaggle/input'):")
        print("      for f in files:")
        print("          if 'inference' in f.lower() or 'server' in f.lower():")
        print("              print(os.path.join(root, f))")

else:
    print("Not in competition mode - skipping inference server setup.")
    print("Local testing: use generate() or predict() directly.")
    
    # Quick test of predict function
    test_q = "What is 2 + 2?"
    print(f"\nTest predict('{test_q}'):")
    print(predict(test_q)[:200] + "..." if len(predict(test_q)) > 200 else predict(test_q))